In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision as tv
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd

plt.ion()   # interactive mode

In [4]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
data_transforms = {
    'train': transforms.Compose([
                      transforms.Resize((95,95)),
                      transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5 ), (0.5, 0.5, 0.5 )),
                      
                     
                                                      ])

}

data_dir = "train_images"

image_datasets = {x: datasets.ImageFolder(data_dir, data_transforms[x]) for x in ['train']}


trainloader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=24,
                                             shuffle=True, num_workers=4)
              for x in ['train']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}
class_names = image_datasets['train'].classes


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(16928, 121)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

net = CNN()

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
print(device)

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [9]:
for epoch in range(1):  # loop over the dataset multiple times
#     running_loss = 0.0
    for i, data in enumerate(trainloader['train'], 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

print('Finished Training')

Finished Training


In [10]:
import torch
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method that dataloader calls
    def __getitem__(self, index):

        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        
        image_name = path.split('/')[-1]

        return img, target, image_name

    def __len__(self):
        return len(self.imgs)

In [11]:
data_transforms_TEST = {
    'test': transforms.Compose([
                      transforms.Resize((95,95)),
                      transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5 ), (0.5, 0.5, 0.5 ))
                               ])

}

data_dir_TEST = 'test_images'

image_datasets_TEST = {x: ImageFolderWithPaths(data_dir_TEST, data_transforms_TEST[x]) for x in ['test']}


testloader = {x: torch.utils.data.DataLoader(image_datasets_TEST[x], batch_size=4,
                                             shuffle=False, num_workers=4)
              for x in ['test']}
dataset_sizes_TEST = {x: len(image_datasets_TEST[x]) for x in ['test']}

In [12]:
image_classes = []
image_names = []

with torch.no_grad():
    for data in testloader['test']:
        images, labels, path = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        image_classes.extend(predicted.tolist())
        img_name = list(path)
        image_names.extend(img_name)
        
predictions = pd.DataFrame({'image': image_names, 'class': image_classes})

In [13]:
predictions.to_csv('output_NEWEST1.csv', index=False)